In [1]:
from IPython.core.display import HTML, display
from string import Template
import pandas as pd
import json, random

In [2]:
HTML("<script src='d3.js'></script>")

In [3]:
df = pd.read_csv("6_convocation.csv")

In [4]:
df.head()

,full_name,date_of_birth,place_of_birth,gender,party
0,Айсина Майра Араповна,10.11.1956,Акмолинская область,ж,Нур Отан
1,Аманжолова Зауреш Джуманалиевна,15.03.1958,Тараз,ж,Нур Отан
2,Аронова Ирина Петровна,05.08.1957,Калининградская область (РФ),ж,Нур Отан
3,Ахметов Сапар Кайратович,27.10.1965,Северо-Казахстанская область,м,Нур Отан
4,Абдраим Бакытжан Жарылкасынович,13.05.1965,Жамбылская область,м,Нур Отан


In [5]:
df_dict = df.to_dict(orient="records")

In [6]:
js_text_template = Template('''

var margin = {top: 20, right: 20, bottom: 70, left: 40},
    width = 600 - margin.left - margin.right,
    height = 300 - margin.top - margin.bottom;

// Parse the date / time
var parseDate = d3.time.format("%d.%m.%Y").parse;

var xScale = d3.scale.ordinal().rangeRoundBands([0, width], .05);

var yScale = d3.scale.linear().range([height, 0]);

var xAxis = d3.svg.axis()
    .scale(xScale)
    .orient("bottom");
    //.tickFormat(d3.time.format("%Y-%m"));

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left")
    .ticks(10);

var svg = d3.select("#$maid").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", 
          "translate(" + margin.left + "," + margin.top + ")");
          
          
var data = $python_data;

data.forEach(function(d) {
    d.age = parseDate(d.date_of_birth);
    var diff_ms = Date.now() - d.age.getTime();
    var age_dt = new Date(diff_ms); 
    d.age =  Math.abs(age_dt.getUTCFullYear() - 1970);
});

var ageList = {};
data.forEach(function(i) { ageList[i.age] = (ageList[i.age]||0) + 1;});

var my_data = [];
for (const [key, value] of Object.entries(ageList)) {
  my_data.push({age: key, count: value});
}



xScale.domain(my_data.map(function(d) { return d.age; }));
yScale.domain([0, d3.max(my_data, function(d) { return d.count; })]);

svg.append("g")
  .attr("class", "x axis")
  .attr("transform", "translate(0," + height + ")")
  .call(xAxis)
  .selectAll("text")
  .style("text-anchor", "end")
  .attr("dx", "-.8em")
  .attr("dy", "-.55em")
  .attr("transform", "rotate(-90)" );

svg.append("g")
  .attr("class", "y axis")
  .call(yAxis)
.append("text")
  .attr("transform", "rotate(-90)")
  .attr("y", 6)
  .attr("dy", ".71em")
  .style("text-anchor", "end")
  .text("Сумма");

svg.selectAll("bar")
  .data(my_data)
  .enter().append("rect")
  .style("fill", "steelblue")
  .attr("x", function(d) { return xScale(d.age); })
  .attr("width", xScale.rangeBand())
  .attr("y", function(d) { return yScale(d.count); })
  .attr("height", function(d) { return height - yScale(d.count); });

  
var line_poists = d3.svg.line()
    .x(function(d) { return xScale(d.age) + xScale.rangeBand()/2 ; })
    .y(function(d) { return yScale(d.count); });

var linegraph = svg.selectAll("path.line").data([my_data], function(d) { return d.age; });

linegraph.enter().append("path")
.attr("d", line_poists);         

''');

In [7]:
js_text = js_text_template.substitute({
    "python_data":json.dumps(df_dict),
    "maid":"scatter-id"
})


html_template = Template('''
    <style>
    .dot{
        stroke: #000;
    }
    
    .legend{
        color:blue;
    }
    
    .axis path,
    .axis line{
        fill: none;
        shape-rendering:crispEdges;
    
    }
    
    path { 
    fill: none; 
    stroke: green; 
    }
    
    </style>
    <div id="scatter-id"></div>
    <script>$js_text</script>
''')

In [8]:
HTML(html_template.substitute({'js_text':js_text}))